# Objective <a class="anchor" id="inicio"></a>

In this notebook, we will explore a way to implement tabular Temporal Difference (TD) methods, which we will use to solve a variety of task environments. Multiple experiments will be conducted to evaluate the strengths of each algorithm across different settings.

This is a complementary material for the [slides on temporal difference](https://github.com/EAndrade-Lotero/BasicTabularRL/blob/main/slides/BasicTabularRL.pdf). I recommend reading these first and then come back to this notebook.

This notebook is based on the presentation by Sanghi (2021), Chapter 5, and its [notebooks](https://github.com/Apress/deep-reinforcement-learning-python), as well as Sutton and Barto (2018), Chapter 6.

[Go to Exercise 1](#ej1)

# Dependencies

When starting the notebook or restarting the kernel, you can load all the dependencies for this notebook by running the following cells. This is also the place to install any missing dependencies.

**From Python:**

In [ ]:
# Uncomment to install requirements in linux or mac
#!pip3 install -r requirements.txt
#!pip3 install gymnasium[toy-text]

# Uncomment to install requirements in windows
#!python -m pip install -r requirements.txt
#!python -m pip installgymnasium[toy-text]

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored, cprint

**For this notebook:**

In [ ]:
from ambientes import GridworldEnv, CliffworldEnv
from agents import Agent
from algoritmos import *
from utils import Episode, Experiment
from plot_utils import PlotGridValues, Plot
from tests import *

# Sections

Our plan is as follows:

* [Method for policy evaluation](#eval)
* [Control methods](#control)

# Method for policiy evaluation <a class="anchor" id="eval"></a>
    
([Back to top](#inicio))

Let us recall that the first step in solving a task environment is addressing the evaluation problem. Here, the goal is to determine the value of a state given a policy. In other words, we want to estimate the expected value of the total discounted reward when following a policy starting from a state $s$. This needs to be done for each state $s$ in the task environment. We will examine the Temporal Difference (TD) method to tackle this problem, using the GridWorld environment as an example.

We begin by defining a policy for a $4\times 4$ grid world (note that this is the same policy we worked with in the previous notebook):

In [ ]:
shape = (4,4)
env = GridworldEnv(shape=shape)
policy = ([env.NORTH] + [env.EAST]*(shape[0] - 2) + [env.SOUTH]) * shape[1]
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
pp.plot_policy(policy)

We already know the state values. We will use the dynamic programming method to compute them:

In [ ]:
V = policy_eval(env, policy)
pp.plot_policy_and_values(policy, V)

In the Temporal Difference (TD) evaluation method, it is not necessary to wait for the entire episode to end before updating our value estimates for the visited states. Instead, at each step, we update the value estimate of the most recently visited state by bootstrapping with the values stored in memory. The pseudocode for the algorithm is as follows:

<img src="./imagenes/td_evaluationb.png" width="500"/>

<a class="anchor" id="ej1"></a>**Exercise 1:**  

([Next exercise](#ej2))

Implement lines 5 to 8 from the pseudocode above in the following code block:

In [ ]:
def td_0_evaluation(env, policy:np.array, alfa:float=0.1, gama:float=1, max_iter:int=500, max_steps=1000, V:np.array=None) -> np.array:
    '''
    Método de diferencia temporal para estimar el valor de los 
    estados de un MDP generando una muestra de episodios con base en una política dada. 
    Input:
        - env, un ambiente con atributos nA, nS, shape 
               y métodos reset(), step()
        - policy, una política determinista, policy[state] = action
        - alfa, real con el parámetro de step-size
        - gama, con el parámetro de factor de descuento
        - max_iter, entero con la cantidad máxima de episodios
        - max_steps, entero con la cantidad máxima de pasos
        - Opcional: V, un np.array que por cada s devuelve su valor estimado
    Output:
        - V, un np.array que por cada s devuelve su valor estimado
    '''
    if V is None:
        V = np.zeros(env.nS)
    for _ in range(max_iter):
        state = np.random.randint(env.nS)
        env.state = state
        done = False
        counter = 0
        while not done:
            pass
            # AQUÍ SU CÓDIGO
            
            # HASTA AQUÍ SU CÓDIGO
            counter += 1
            if counter > max_steps:
                break
    return V 


Check your answer using the following cell:

In [ ]:
np.random.seed(3)
V = td_0_evaluation(env, policy)
VV = np.flipud(np.reshape(V, shape))
test = np.array([[ 0.,         -4.36081499, -3.97308306, -2.99943748],
                 [-0.99993144, -3.77424704, -2.99432731, -1.99999999],
                 [-1.99314462, -2.7058518,  -1.99561459, -1.   ],
                 [-2.85262296, -1.7706891, -0.99695675,  0.   ]])
assert(np.all(np.isclose(VV, test)))
print('¡Correcto!')

---

## Control Methods <a class="anchor" id="control"></a>

([Back to top](#inicio))

Solving an environment involves finding the optimal policy that the agent should follow to maximize its utility. In the grid world, the optimal policy is as follows:

In [ ]:
shape = (4,4)
env = GridworldEnv(shape=shape)
policy = value_iteration(env, discount_factor=1, theta=0.01, verbose=0)
print('Política óptima:')
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
pp.plot_policy(policy)

To find the optimal policy, the Temporal Difference (TD) method maintains an estimate of the state-action value for the policy in memory. Each time this estimate is updated, the policy is also improved using an $\epsilon$-greedy improvement strategy:

$$
\pi_{k+1}(a|s) = \begin{cases}
1-\epsilon &\text{if }a=\text{arg}\!\max_{a'} Q_{\pi_k}(s,a') \cr
\frac{\epsilon}{\#\text{actions}-1} &\text{otherwise}
\end{cases}
$$

### The `Episode` Class

To run simulations in an organized and straightforward manner, we have implemented the `Episode` class, which is located in the `utils.py` module. Let's observe a random algorithm acting in the grid world environment:

In [ ]:
# Create environment
shape = (3,3)
env = GridworldEnv(shape=shape)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
}
agent = Agent(parameters=parameters)
# Create episode
episode = Episode(environment=env, \
                  env_name='GW', \
                  agent=agent, \
                  model_name='Random', \
                  num_rounds=3, \
                )
# Run and show information
df = episode.run(verbose=4)

Notice that we have executed the `run()` method, which runs a single episode, with the `verbose=4` option. This setting causes the method to print detailed information for each step of the episode. If we do not need this information, we can set `verbose=0` (or omit this argument entirely, as it is optional with a default value of 0) to avoid unnecessary delays in data processing.

Now, let’s examine a graphical representation of the evolution of total rewards over multiple episodes:

In [ ]:
# Create environment
shape = (4,4)
env = GridworldEnv(shape=shape)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
}
agent = Agent(parameters=parameters)
# Create episode
episode = Episode(environment=env, \
                  env_name='GW', \
                  agent=agent, \
                  model_name='Random', \
                  num_rounds=100
                )
# Train agent
df = episode.simulate(num_episodes=50, verbose=0)
# Plot rewards
Plot(df).plot_rewards()

Here we observe the evolution of rewards over 50 episodes, each consisting of a maximum of 100 steps. For this, we used the `simulate()` method.

We can inspect the agent’s policy by visualizing its `policy` attribute:

In [ ]:
p = episode.agent.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
print('Política del agente:')
pp.plot_policy(policy)

**The `Experiment` Class**

One way to use the `Experiment` class is to test the already-trained agent without requiring further exploration. We can apply the `run_experiment()` method on the agent and plot a histogram of the total rewards per episode:

In [ ]:
# Create experiment
experiment = Experiment(environment=env,\
                        env_name='GW', \
                        num_rounds=10, \
                        num_episodes=10, \
                        num_simulations=10)
# Test agent
agents = experiment.run_experiment(agents=[agent],\
                                  names=['Random'], \
                                  measures=['hist_reward'], \
                                  learn=False)

### Temporal Difference Control <a class="anchor" id="TDcontrol"></a>

([Back to Control](#control))

We will implement two agents: one using the SARSA learning rule and the other using the Q-learning rule.

<a class="anchor" id="ej2"></a>**Exercise 2:**  

([Previous exercise](#ej1)) ([Next exercise](#ej3))

Implement the SARSA agent according to the SARSA agent pseudocode:

<img src="./imagenes/sarsa_agent.png" width="500"/>

Use the following cell to implement the agent. The focus should be on implementing line 5, where you calculate the update using bootstrapping (estimate), the temporal difference error (delta), and then update the previous value by moving it toward delta by a fraction $\alpha$.

In [ ]:
class SARSA(Agent) :
    '''
    Implements a SARSA learning rule.
    '''

    def __init__(self, parameters:dict):
        super().__init__(parameters)
        self.alpha = self.parameters['alpha']
        self.debug = False
   
    def update(self, next_state, reward, done):
        '''
        Agent updates its model.
        '''
        # obtain previous state
        state = self.states[-1]
        # obtain previous action
        action = self.actions[-1]
        # Get next_action
        next_action = self.make_decision()
        # Find bootstrap
        estimate = ... # recompensa más descuento por valor del siguiente estado
        # Obtain delta
        delta = ... # Diferencia temporal: estimado menos valor del estado actual
        # Update Q value
        prev_Q = self.Q[state, action]
        self.Q[state, action] = ... # Actualizar en la dirección de delta por una fracción alfa
        # Update policy
        self.update_policy(state)
        if self.debug:
            print('')
            print(dash_line)
            print(f'Learning log:')
            print(f'state:{state}')
            print(f'action:{action}')
            print(f'reward:{reward}')
            print(f'estimate:{estimate}')
            print(f'Previous Q:{prev_Q}')
            print(f'delta:{delta}')
            print(f'New Q:{self.Q[state, action]}')


Check your code by running the following cell:

In [ ]:
shape = (3,4)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
    'alpha': 0.1, \
}
agent_SARSA = SARSA(parameters=parameters)
test = test_sarsa(agent_SARSA)
if test:
    cprint(colored('¡Test superado!', 'green'))
else:
    cprint(colored('¡Implementación incorrecta!', 'red'))

---

<a class="anchor" id="ej3"></a>**Exercise 3:**  

([Previous exercise](#ej2)) ([Next exercise](#ej4))

Implement the agent using the Q-learning rule, following the pseudocode below:

<img src="./imagenes/q_learning_agent.png" width="450"/>

In [ ]:
class Q_learning(Agent) :
    '''
    Implements a Q-learning rule.
    '''

    def __init__(self, parameters:dict):
        super().__init__(parameters)
        self.alpha = self.parameters['alpha']
        self.debug = False
   
    def update(self, next_state, reward, done):
        '''
        Agent updates its model.
        '''
        # obtain previous state
        state = ... # Aquí estado previo
        # obtain previous action
        action = self.actions[-1]
        # Find bootstrap
        maxQ = self.max_Q(next_state) 
        estimate = ... # Calcula el estimado
        # Obtain delta
        delta = ... # Calcula el delta
        # Update Q value
        prev_Q = self.Q[state, action]
        self.Q[state, action] = ... # Actualiza el valor
        # Update policy
        self.update_policy(...) # Actualizar la política en el estado        
        if self.debug:
            print('')
            print(dash_line)
            print(f'Learning log:')
            print(f'state:{state}')
            print(f'action:{action}')
            print(f'reward:{reward}')
            print(f'estimate:{estimate}')
            print(f'Previous Q:{prev_Q}')
            print(f'delta:{delta}')
            print(f'New Q:{self.Q[state, action]}') 

Corra la siguiente celda para verificar su implementación.

In [ ]:
shape = (3,4)
# Create agent
parameters = {\
    'nS': np.prod(shape),\
    'nA': 4,\
    'gamma': 1,\
    'epsilon': 0.1,\
    'alpha': 0.1, \
}
agent_Q = Q_learning(parameters=parameters)
test = test_q(agent_Q)
if test:
    cprint(colored('¡Test superado!', 'green'))
else:
    cprint(colored('¡Implementación incorrecta!', 'red'))

---

<a class="anchor" id="ej4"></a>**Exercise 4:**  

([Previous exercise](#ej3)) ([Next exercise](#ej5))

Compare the performance of the SARSA agent with the Q-learning agent in the cliff walking environment (implemented in the `CliffworldEnv` class within the `ambientes` module). Recall that this example was discussed in class, where it was mentioned that Q-learning does not account for occasional slips in the $\epsilon$-greedy policy, whereas SARSA does.

Use the following specifications:

* Maximum number of steps: 200  
* Number of episodes: 500  
* Number of simulations: 10  

---

We can observe the resulting policies for the agents by running the following cells:

**Note:** The following cells will only work after completing Exercise 4.

In [ ]:
# Create environment
shape = (3,4)
env = CliffworldEnv(shape=shape)
shape = (3,4)
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
sarsa = agents[0]
p = sarsa.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
policy = np.flipud(np.reshape(policy, shape))
pp.plot_policy(policy)

In [ ]:
# Create environment
shape = (3,4)
env = CliffworldEnv(shape=shape)
shape = (3,4)
pp = PlotGridValues(shape=shape, dict_acciones=env.dict_acciones)
q_agent = agents[1]
p = q_agent.policy
policy = [np.argmax(p[s,]) for s in range(env.nS)]
policy = np.flipud(np.reshape(policy, shape))
pp.plot_policy(policy)

During the learning process, we want the agent to explore different courses of action to gain greater confidence in finding an optimal policy. However, when deploying the agent in production, we want it to perform at its best. To achieve this, we need to set its $\epsilon$ parameter to 0.

Let us compare the optimal performance of the SARSA and Q-learning agents without exploration.

In [ ]:
# Shut down exploration
agents[0].epsilon = 0
agents[1].epsilon = 0
for s in range(env.nS):
    agents[0].update_policy(s)
    agents[1].update_policy(s)

In [ ]:
# Create experiment
experiment = Experiment(environment=env,\
                 env_name='Cliff', \
                 num_rounds=100, \
                 num_episodes=100, \
                 num_simulations=1)
# Use stored agents to run test
experiment.run_experiment(
                agents=agents,\
                names=['SARSA', 'Q_learning'], \
                measures=['hist_reward'],\
                learn=False)
print('¡Listo!')

---

# Key Takeaways

* Evaluate a policy using Temporal Difference methods.  
* Improve a policy using SARSA and Q-learning.  

# References

([Back to top](#inicio))

Shanghi, N. (2021) Deep Reinforcement Learning with Python: With PyTorch, TensorFlow and OpenAI Gym. Apress. 

Sutton R., & Barto, A., (2015) Reinforcement Learning: An Introduction, 2nd Edition. A Bradford Book. Series: Adaptive Computation and Machine Learning series. 

Winder, P., (2021) Reinforcement Learning: Industrial Applications of Intelligent Agents. O’Relly.